In [ ]:
# MySQL code:

USE sakila;
SELECT * FROM rental;
DROP TABLE IF EXISTS rentals_may;
CREATE TABLE rentals_may(
rental_id INT(11) UNIQUE NOT NULL,
rental_date DATETIME DEFAULT NULL,
inventory_id INT(11) DEFAULT NULL,
customer_id INT(11) DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id INT(11) DEFAULT NULL,
last_update TIMESTAMP DEFAULT NULL,
CONSTRAINT PRIMARY KEY(rental_id)
);
INSERT INTO rentals_may (SELECT * FROM rental WHERE MONTH(rental_date) = 5);
SELECT * FROM rentals_may;

DROP TABLE IF EXISTS rentals_june;
CREATE TABLE rentals_june(
rental_id INT(11) UNIQUE NOT NULL,
rental_date DATETIME DEFAULT NULL,
inventory_id INT(11) DEFAULT NULL,
customer_id INT(11) DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id INT(11) DEFAULT NULL,
last_update TIMESTAMP DEFAULT NULL,
CONSTRAINT PRIMARY KEY(rental_id)
);
INSERT INTO rentals_june (SELECT * FROM rental WHERE MONTH(rental_date) = 6);
SELECT * FROM rentals_june;

SELECT customer_id, COUNT(inventory_id) AS number_of_rentals FROM rentals_may GROUP BY customer_id;

SELECT customer_id, COUNT(inventory_id) AS number_of_rentals FROM rentals_june GROUP BY customer_id;

Lab | SQL Queries 9
In this lab, you will be using the Sakila database of movie rentals. You have been using this database for a couple labs already, but if you need to get the data again, refer to the official installation link.

The database is structured as follows: DB schema

Instructions: 

In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.

Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes: Check the number of rentals for each customer for May Check the number of rentals for each customer for June Hint: You can store the results from the two queries in two separate dataframes.

In [2]:
# Retrieve the data
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [4]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [6]:
query = 'SELECT * FROM rentals_may'
rentals_may = pd.read_sql_query(query, engine)
rentals_may.head()


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [10]:
query = 'SELECT customer_id, COUNT(inventory_id) AS number_of_rentals FROM rentals_may GROUP BY customer_id;'
number_rentals_may = pd.read_sql_query(query, engine)
number_rentals_may.head()

,customer_id,number_of_rentals
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5


In [11]:
query = 'SELECT * FROM rentals_june'
rentals_june = pd.read_sql_query(query, engine)
rentals_june.head()


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [12]:
query = 'SELECT customer_id, COUNT(inventory_id) AS number_of_rentals FROM rentals_june GROUP BY customer_id;'
number_rentals_june = pd.read_sql_query(query, engine)
number_rentals_june.head()

,customer_id,number_of_rentals
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6


Write a function that checks if customer borrowed more or less movies in the month of June as compared to May.
Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the merge function.

In [34]:
data = number_rentals_may.merge(number_rentals_june, left_on='customer_id', right_on='customer_id')
data.head()

,customer_id,number_of_rentals_x,number_of_rentals_y
0,130,2,6
1,459,1,7
2,408,3,3
3,333,1,4
4,222,5,2


In [51]:
may = 0
june = 0
for value in range(len(data['customer_id'])):    
    if data.number_of_rentals_x[value] > data.number_of_rentals_y[value]:
        may += 1
    else:
        june +=1
            
print ('On may,',may,'customers made more rentals');
print ('On june,',june,'customers made more rentals');

if may > june:
    print( 'May wins!')
else:
    print('June wins!')
    

On may, 92 customers made more rentals
On june, 420 customers made more rentals
June wins!
